In [ ]:
# Preload the interleaf data 
import pandas as pd
df = pd.read_csv('/home/ubuntu/latest_llava/LLaVA/playground/data/dpo/1231-13b-selfsamples/lrv_1699__self-sampled_at_least2_in_diff_for_dpo_inference1230.csv')

In [ ]:
grouped = df.groupby(['image', 'question'])
filtered_df = grouped.filter(lambda x: len(x) == 4)
len(filtered_df), len(df)

In [ ]:
df['Output'] = df['cur_candidate_answer']
del df['cur_candidate_answer']

# The following turns it into SteerLM format

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your original DataFrame
new_rows = []

# Group the DataFrame by 'image'
for image, group in df.groupby(['image', 'question']):
    # Find the 'GPT4V_response' and candidate responses within this group
    candidate_responses = group
    # If there are no candidate responses, skip this group
    if candidate_responses.empty:
        continue
    
    # Calculate the difference in the total sum of 'Helpfulness', 'Correctness', and 'Coherence'
    candidate_responses['Sum_quality'] = candidate_responses['Helpfulness'] + candidate_responses['Correctness'] + candidate_responses['Coherence']
    diff = abs(max(candidate_responses['Sum_quality']) - min(candidate_responses['Sum_quality']))
    
    # If the difference is too small, skip this group
    if diff < 2:  # Define your threshold
        continue
    
    # Find the response with the highest 'Sum_quality'
    max_row_index = candidate_responses['Sum_quality'].idxmax()
    max_row = candidate_responses.loc[max_row_index]
    
    # Filter for responses that are at least 2 lesser or equal
    filtered_responses = candidate_responses[candidate_responses['Sum_quality'] <= max_row['Sum_quality'] - 2]
    
    # If the filtered group is empty, skip this group
    if filtered_responses.empty:
        continue
    
    # Randomly sample one response from the filtered group
    selected_row = filtered_responses.sample(n=1)
    assert max_row['prompt'] == selected_row['prompt'].item()
    # Create a new row with the 'GPT4V_response' and the selected candidate response
    new_row = {
        'image': max_row['image'],
        'question': max_row['prompt'],
        # 'GPT4V_question_and_response': max_row['conversations'],  # Add 'conversations' column
        'chosen': max_row['Output'],
        # 'Candidate_question_and_response': selected_row['conversations'],  # Add 'conversations' column
        'rejected': selected_row['Output'].item(),
        'Helpfulness_gpt4v': max_row['Helpfulness'],
        'Correctness_gpt4v': max_row['Correctness'],
        'Coherence_gpt4v': max_row['Coherence'],
        'Sum_of_helpfulness_correctness_coherence_gpt4v': max_row['Helpfulness'] + max_row['Correctness'] + max_row['Coherence'],
        'Complexity_gpt4v': max_row['Complexity'],
        'Verbosity_gpt4v': max_row['Verbosity'],
        'Helpfulness_candidate': selected_row['Helpfulness'].values[0],
        'Correctness_candidate': selected_row['Correctness'].values[0],
        'Coherence_candidate': selected_row['Coherence'].values[0],
        'Sum_of_helpfulness_correctness_coherence_candidate': selected_row['Helpfulness'].values[0] + selected_row['Correctness'].values[0] + selected_row['Coherence'].values[0],
        'Complexity_candidate': selected_row['Complexity'].values[0],
        'Verbosity_candidate': selected_row['Verbosity'].values[0],
    }
    
    new_rows.append(new_row)

new_df = pd.DataFrame(new_rows)
sum_attributes = ['Sum_of_helpfulness_correctness_coherence_gpt4v', 
 'Sum_of_helpfulness_correctness_coherence_candidate']
new_df[sum_attributes].describe()

def form_multi_turn(row):
    first_turn = {'from': 'human', 'value': row['question']}
    
    first_candidate = {'from': 'gpt', 'value': row['chosen']}
    candidate_attribute_str = f'''The previous answer is in the following quality metrics in the 0-4 likert scale: Helpfulness: {row['Helpfulness_candidate']}, Correctness: {row['Correctness_candidate']}, Coherence: {row['Coherence_candidate']}, Complexity: {row['Complexity_candidate']}, Verbosity: {row['Verbosity_candidate']}.
    Please answer the question such that your response is in the following attributes in 5-point Likert scale: Helpfulness: {row['Helpfulness_gpt4v']}, Correctness: {row['Correctness_gpt4v']}, Coherence: {row['Coherence_gpt4v']}, Complexity: {row['Complexity_gpt4v']}, Verbosity: {row['Verbosity_gpt4v']}. '''
    preference = {'from': 'human', 'value': candidate_attribute_str}
    final_turn = {'from': 'gpt', 'value': row['rejected']}
    return [first_turn, first_candidate, preference, final_turn]

turn_into_steerlm = False
if turn_into_steerlm:
    new_df['conversations'] = new_df.apply(form_multi_turn, axis=1)
    new_df['image'] = 'scigraph/' + new_df['image'].astype(str)
    new_df.to_json('/home/ubuntu/latest_llava/LLaVA/playground/data/concatnateed_helpsteer/scigraph3000_self-sampling.json', orient='records')
# new_df['conversations'] = new_df.apply(form_multi_turn, axis=1)
# new_df['image'] = 'scigraph/' + new_df['image'].astype(str)
# new_df.to_json('/home/ubuntu/latest_llava/LLaVA/playground/data/concatnateed_helpsteer/scigraph3000_self-sampling.json', orient='records')

In [ ]:
new_df['chosen'][0]

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
intermediate_csv = '/home/ubuntu/latest_llava/LLaVA/playground/data/dpo/lrv-10k-13k-sampled_at_least2_in_diff_for_dpo_inference0104.csv'

# The start of a json


In [ ]:
import pandas as pd

intermediate_csv = '/home/ubuntu/latest_llava/LLaVA/playground/data/dpo/lrv-10k-13k-sampled_at_least2_in_diff_for_dpo_inference0104.csv'

df = pd.read_csv(intermediate_csv)
print(df)


In [ ]:
import torch
import os
import json
import pandas as pd
from tqdm import tqdm
import shortuuid

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, process_images, load_image_from_base64, get_model_name_from_path
from llava.train.train import preference_collator_fn
from functools import partial
import tqdm
from llava.train.llava_trainer import get_batch_logps
from PIL import Image
import math
from llava.train.train import encode_multimodal_preference_sample
from datetime import date
import torch.utils.data as torch_data
disable_torch_init()
model_path = '/home/ubuntu/latest_llava/LLaVA/checkpoints/llava-v1.5-13b'
# model_base = '/home/ubuntu/latest_llava/LLaVA/checkpoints/llava-v1.5-13b'
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, None, model_name)

use_im_start_end= False # Default setting from llava 1.5 




In [ ]:
IMAGE_FOLDER="/home/ubuntu/latest_llava/llava_1dot5data/coco/train2017/"
intermediate_csv = '/home/ubuntu/latest_llava/LLaVA/playground/data/dpo/lrv-10k-13k-sampled_at_least2_in_diff_for_dpo_inference0104.csv'
import pandas as pd
df = pd.read_csv(intermediate_csv)
intermediate_json = intermediate_csv.split('csv')[0] + 'json'


df.to_json(intermediate_json, orient='records')

def _convert_to_llava_answer_turn(answer):
    return {"from": "gpt", "value": answer}

def _convert_to_llava_question_turn(question):
    return {"from": "human", "value": "<image>\n" + question}

# def _convert_to_llava_question_turn(question):
#     return {"from": "human", "value": question}

df['question'] = df['question'].apply(_convert_to_llava_question_turn)
df['chosen'] = df['chosen'].apply(_convert_to_llava_answer_turn)
df['rejected'] = df['rejected'].apply(_convert_to_llava_answer_turn)

df.to_json(intermediate_json, orient='records')

# FILEPATH: /home/ubuntu/latest_llava/LLaVA/llava/eval/eval_logp_for_ddp_self_sampled_llava8times.ipynb
output_path = intermediate_csv.split('csv')[0] + f'with_logp{model_name}_{date.today()}.json'

In [ ]:
from llava.train.train import DPODataset
multimodal_cfg = {
    'image_processor': image_processor,
    'is_multimodal': True,
    # 'image_token_len': image_token_len, # TODO check if needed
    'use_im_start_end': use_im_start_end,
    'image_aspect_ratio': 'pad', 
    'data_path': intermediate_json,
    'image_folder': IMAGE_FOLDER,
}
dpo_dataset = DPODataset(tokenizer, multimodal_cfg)

In [ ]:
dpo_dataset[0][0].keys()

In [ ]:
dpo_dataset[0][0]['labels'].shape

In [ ]:
len(dpo_dataset)

In [ ]:
from llava.train.train import DataCollatorForDPODataset

data_collator = DataCollatorForDPODataset(tokenizer, beta=0.1, mod_token_weight=None)

In [ ]:
dataloader = torch_data.DataLoader(dpo_dataset, batch_size=1, collate_fn=collate_fn,
                                    num_workers=5, shuffle=False)

In [ ]:
with torch.no_grad():
    logps = get_batch_logps(model, dataloader, data_collator, device='cuda')

In [ ]:
input_ids.shape, labels.shape

In [ ]:
output_logits = output.logits

In [ ]:
output_logits_cleaned = output_logits[:, 575:]
output_logits_cleaned

In [ ]:
output_logits_cleaned.shape

In [ ]:
labels.shape

In [ ]:
win_logp_list = []
rej_logp_list = []

win_avg_logp_list = []
rej_avg_logp_list = []

win_per_token_logp_list = []
rej_per_token_logp_list = []

collate_fn = partial(preference_collator_fn, pad_token_id=tokenizer.pad_token_id)
dataloader = torch_data.DataLoader(dpo_dataset, batch_size=1, collate_fn=collate_fn,
                                    num_workers=5, shuffle=False)
with torch.inference_mode():
    for batch in tqdm.tqdm(dataloader):
        for key in ['win', 'rej']:
            input_ids = batch[f'{key}_input_ids'].cuda()
            labels = batch[f'{key}_labels'].cuda()
            attention_mask = batch[f'{key}_attention_mask'].cuda()

            output = model(
                input_ids=input_ids,
                labels=labels,
                attention_mask=attention_mask,
                images=batch['images'].half().cuda()
            )
            per_token_logp, log_prob, average_log_prob = get_batch_logps(output.logits[:, 575:], labels, return_all=True)

            # print(per_token_logp.shape, input_ids.shape, labels.shape, flush=True)
            assert per_token_logp.size(1) >= input_ids.size(1) - 1
            per_token_logp = per_token_logp.tolist()
            # per_token_logp = [x[:input_ids[i].ne(tokenizer.pad_token_id).sum().item()] for i, x in enumerate(per_token_logp)]
            log_prob = log_prob.tolist()
            average_log_prob = average_log_prob.tolist()

            if key == 'win':
                win_logp_list += log_prob
                win_avg_logp_list += average_log_prob
                win_per_token_logp_list += per_token_logp
            else:
                rej_logp_list += log_prob
                rej_avg_logp_list += average_log_prob
                rej_per_token_logp_list += per_token_logp
            # print(f'{key} logits in {output.logits.shape}, logp in {log_prob.shape} avg_logp in {average_log_prob.shape}')


In [ ]:
df = pd.read_json(intermediate_json)
def _convert_to_llava_answer_turn(answer):
    return {"from": "gpt", "value": answer}
# Add each list as a column to the dataframe with its variable name
df['win_logp'] = win_logp_list
df['rej_logp'] = rej_logp_list
df['win_avg_logp'] = win_avg_logp_list
df['rej_avg_logp'] = rej_avg_logp_list
df['win_per_token_logp'] = win_per_token_logp_list
df['rej_per_token_logp'] = rej_per_token_logp_list
exisitng_columns = ['rej_logp', 
                    'win_logp',
                    'rej_avg_logp',
                    'win_avg_logp',
                    'rej_per_token_logp',
                    'win_per_token_logp']
for col in exisitng_columns:
    rename = 'ref_' + col
    df[rename] = df[col]
    del df[col]
# df['chosen'] = df['chosen'].apply(_convert_to_llava_answer_turn)
# df['rejected'] = df['rejected'].apply(_convert_to_llava_answer_turn)
# def _convert_to_llava_question_turn(question):
#     return {"from": "human", "value": question}
# df['question'] = df['question'].apply(_convert_to_llava_question_turn)
df.to_json(output_path, orient='records')

In [ ]:
# import torch
# import os
# import json
# import pandas as pd
# from tqdm import tqdm
# import shortuuid

# from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
# from llava.conversation import conv_templates, SeparatorStyle
# from llava.model.builder import load_pretrained_model
# from llava.utils import disable_torch_init
# from llava.mm_utils import tokenizer_image_token, process_images, load_image_from_base64, get_model_name_from_path
# from llava.train.train import preference_collator_fn
# from functools import partial
# import tqdm
# from llava.train.llava_trainer import get_batch_logps
# from PIL import Image
# import math
# from llava.train.train import encode_multimodal_preference_sample
# from datetime import date
# import torch.utils.data as torch_data
# disable_torch_init()
# model_path = '/home/ubuntu/latest_llava/LLaVA/checkpoints/llava-v1.5-13b'
# # model_base = '/home/ubuntu/latest_llava/LLaVA/checkpoints/llava-v1.5-13b'
# model_name = get_model_name_from_path(model_path)
# tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, None, model_name)

# use_im_start_end= False # Default setting from llava 1.5 

# IMAGE_FOLDER="/home/ubuntu/latest_llava/llava_1dot5data/coco/train2017/"
# inference_data_path = intermediate_csv
# # FILEPATH: /home/ubuntu/latest_llava/LLaVA/llava/eval/eval_logp_for_ddp_self_sampled_llava8times.ipynb
# output_path = intermediate_csv.split('csv')[0] + f'with_logp{model_name}_{date.today()}.json'

# class PreferenceInferenceDataset(torch_data.Dataset):
#     def __init__(self,
#                  data_path,
#                  tokenizer,
#                  img_processor,
#                  use_im_start_end):
#         self.data = pd.read_csv(data_path)

#         self.mm_cfg = {
#             'image_processor': img_processor,
#             'is_multimodal': True,
#             # 'image_token_len': image_token_len, # TODO check if needed
#             'use_im_start_end': use_im_start_end,
#             'image_aspect_ratio': 'pad'
#         }
#         self.tokenizer = tokenizer

#     def __getitem__(self, index):
#         # After encode multimodal preference sample, 
#         # the image would ahve the pixel values values
#         sample = self.convert_dataframe_row_to_source(self.data.iloc[index])
#         rej_data_dict, win_data_dict = encode_multimodal_preference_sample(sample, self.tokenizer, self.mm_cfg)
#         return rej_data_dict, win_data_dict
    
#     def _get_image(self, img_filename):
#         img_path = os.path.join(IMAGE_FOLDER, img_filename)
#         image = Image.open(img_path).convert('RGB')
#         return image
    

#     def _convert_to_llava_answer_turn(self, answer):
#         return {"from": "gpt", "value": answer}
    
#     def _convert_to_llava_question_turn(self, question):
#         return {"from": "human", "value": "<image>\n" + question}

#     def convert_dataframe_row_to_source(self, row):
#         dict_output = {'question': self._convert_to_llava_question_turn(row['question']), 
#                        # Chosen and rejected based on the sum of the first three quality score
#                     'chosen': self._convert_to_llava_answer_turn(row['chosen']),
#                     'rejected': self._convert_to_llava_answer_turn(row['rejected']),
#                     'image': self._get_image(row['image'])}
#         return dict_output

#     def __len__(self):
#         return len(self.data)
    
preference_torch_dataset = PreferenceInferenceDataset(inference_data_path,
                           tokenizer=tokenizer,
                           img_processor=image_processor,
                           use_im_start_end=False)
preference_torch_dataset[0]

dataset = preference_torch_dataset
collate_fn = partial(preference_collator_fn, pad_token_id=tokenizer.pad_token_id)
dataloader = torch_data.DataLoader(dataset, batch_size=1, collate_fn=collate_fn,
                                    num_workers=5, shuffle=False)
win_logp_list = []
rej_logp_list = []

win_avg_logp_list = []
rej_avg_logp_list = []

win_per_token_logp_list = []
rej_per_token_logp_list = []

with torch.inference_mode():
    for batch in tqdm.tqdm(dataloader):
        for key in ['win', 'rej']:
            input_ids = batch[f'{key}_input_ids'].cuda()
            labels = batch[f'{key}_labels'].cuda()
            attention_mask = batch[f'{key}_attention_mask'].cuda()

            output = model(
                input_ids=input_ids,
                labels=labels,
                attention_mask=attention_mask,
                images=batch['images'].half().cuda()
            )
            per_token_logp, log_prob, average_log_prob = get_batch_logps(output.logits, labels, return_all=True)

            # print(per_token_logp.shape, input_ids.shape, labels.shape, flush=True)
            assert per_token_logp.size(1) >= input_ids.size(1) - 1
            per_token_logp = per_token_logp.tolist()
            # per_token_logp = [x[:input_ids[i].ne(tokenizer.pad_token_id).sum().item()] for i, x in enumerate(per_token_logp)]
            log_prob = log_prob.tolist()
            average_log_prob = average_log_prob.tolist()

            if key == 'win':
                win_logp_list += log_prob
                win_avg_logp_list += average_log_prob
                win_per_token_logp_list += per_token_logp
            else:
                rej_logp_list += log_prob
                rej_avg_logp_list += average_log_prob
                rej_per_token_logp_list += per_token_logp
            # print(f'{key} logits in {output.logits.shape}, logp in {log_prob.shape} avg_logp in {average_log_prob.shape}')

df = pd.read_csv(inference_data_path)
def _convert_to_llava_answer_turn(answer):
    return {"from": "gpt", "value": answer}
# Add each list as a column to the dataframe with its variable name
df['win_logp'] = win_logp_list
df['rej_logp'] = rej_logp_list
df['win_avg_logp'] = win_avg_logp_list
df['rej_avg_logp'] = rej_avg_logp_list
df['win_per_token_logp'] = win_per_token_logp_list
df['rej_per_token_logp'] = rej_per_token_logp_list
exisitng_columns = ['rej_logp', 
                    'win_logp',
                    'rej_avg_logp',
                    'win_avg_logp',
                    'rej_per_token_logp',
                    'win_per_token_logp']
for col in exisitng_columns:
    rename = 'ref_' + col
    df[rename] = df[col]
    del df[col]
df['chosen'] = df['chosen'].apply(_convert_to_llava_answer_turn)
df['rejected'] = df['rejected'].apply(_convert_to_llava_answer_turn)
def _convert_to_llava_question_turn(question):
    return {"from": "human", "value": question}
df['question'] = df['question'].apply(_convert_to_llava_question_turn)
df.to_json(output_path, orient='records')

In [ ]:
679-104

In [ ]:
labels.shape

In [ ]:
output.logits.shape

In [ ]:
input_ids.shape, labels.shape, attention_mask.shape, batch['images'].half().cuda().shape

In [ ]:
output_path

In [ ]:
import matplotlib.pyplot as plt
df[['ref_rej_logp', 'ref_win_logp']].boxplot()

plt.title('13b-007dpo-checkpoint Log Probability on preference dataset before DPO', fontsize=10)

# Set the x-ticks
plt.xticks([1, 2], ['Rejected Responses', 'Preferred Reponses'])
plt.ylabel('Log Probability', fontsize=12)

# Save the plot in high DPI and PDF format
plt.savefig('output.pdf', dpi=300)
plt.show()


In [ ]:
def _convert_to_llava_answer_turn(answer):
    return {"from": "gpt", "value": answer}



In [ ]:
df_reload['chosen'] = df_reload['chosen'].apply(_convert_to_llava_answer_turn)
df_reload['rejected'] = df_reload['rejected'].apply(_convert_to_llava_answer_turn)

In [ ]:
df_reload['rejected']

In [ ]:
df.to_json('/home/ubuntu/LLaVA/playground/data/dpo/dpo_logp.json', orient='records')


# Old draft below

In [ ]:
import torch
import os
import json
import pandas as pd
from tqdm import tqdm
import shortuuid

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, process_images, load_image_from_base64, get_model_name_from_path

from PIL import Image
import math
disable_torch_init()
model_path = '/home/ubuntu/LLaVA/checkpoints/llava-v1.5-13b'
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, None, model_name)

In [ ]:
use_im_start_end= False # Default setting from llava 1.5 

IMAGE_FOLDER='/home/ubuntu/train2017'


In [ ]:
# Inspect previous setup

df_muffin_tsv = '/home/ubuntu/muffin/data/RLHF-V-Hall_v0/RLHF-V-Hall_v0-1837.tsv'
df = pd.read_csv(df_muffin_tsv, sep='\t')

In [ ]:
from llava.train.train import encode_multimodal_preference_sample
import torch.utils.data as torch_data

class PreferenceInferenceDataset(torch_data.Dataset):
    def __init__(self,
                 data_path,
                 tokenizer,
                 img_processor,
                 use_im_start_end):
        self.data = pd.read_csv(data_path, sep='\t')

        self.mm_cfg = {
            'image_processor': img_processor,
            'is_multimodal': True,
            # 'image_token_len': image_token_len, # TODO check if needed
            'use_im_start_end': use_im_start_end,
            'image_aspect_ratio': 'pad'
        }
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        # After encode multimodal preference sample, 
        # the image would ahve the pixel values values
        sample = self.convert_dataframe_row_to_source(self.data.iloc[index])
        rej_data_dict, win_data_dict = encode_multimodal_preference_sample(sample, self.tokenizer, self.mm_cfg)
        return rej_data_dict, win_data_dict
    
    def _get_image(self, img_filename):
        img_path = os.path.join(IMAGE_FOLDER, img_filename)
        image = Image.open(img_path).convert('RGB')
        return image
    

    def _convert_to_llava_answer_turn(self, answer):
        return {"from": "gpt", "value": answer}

    def convert_dataframe_row_to_source(self, row):
        dict_output = {'question': row['question'], 
                       # Chosen and rejected based on the sum of the first three quality score
                    'chosen': self._convert_to_llava_answer_turn(row['chosen']),
                    'rejected': self._convert_to_llava_answer_turn(row['rejected']),
                    'image': self._get_image(row['image'])}
        return dict_output

    def __len__(self):
        return len(self.data)
    
preference_torch_dataset = PreferenceInferenceDataset('/home/ubuntu/muffin/data/RLHF-V-Hall_v0/RLHF-V-Hall_v0-1837.tsv',
                           tokenizer=tokenizer,
                           img_processor=image_processor,
                           use_im_start_end=False)
preference_torch_dataset[0]

In [ ]:
preference_torch_dataset.mm_cfg['image_aspect_ratio']

In [ ]:
def _convert_to_llava_answer_turn(self, answer):
    return {"from": "gpt", "value": answer}

In [ ]:
preference_torch_dataset[0][0]['input_ids'].shape

In [ ]:
from llava.train.train import preference_collator_fn
from functools import partial
import tqdm
from llava.train.llava_trainer import get_batch_logps
dataset = preference_torch_dataset
collate_fn = partial(preference_collator_fn, pad_token_id=tokenizer.pad_token_id)
dataloader = torch_data.DataLoader(dataset, batch_size=1, collate_fn=collate_fn,
                                    num_workers=5, shuffle=False)
win_logp_list = []
rej_logp_list = []

win_avg_logp_list = []
rej_avg_logp_list = []

win_per_token_logp_list = []
rej_per_token_logp_list = []

with torch.inference_mode():
    for batch in tqdm.tqdm(dataloader):
        for key in ['win', 'rej']:
            input_ids = batch[f'{key}_input_ids'].cuda()
            labels = batch[f'{key}_labels'].cuda()
            attention_mask = batch[f'{key}_attention_mask'].cuda()

            output = model(
                input_ids=input_ids,
                labels=labels,
                attention_mask=attention_mask,
                images=batch['images'].half().cuda()
            )
            per_token_logp, log_prob, average_log_prob = get_batch_logps(output.logits, labels, return_all=True)

            # print(per_token_logp.shape, input_ids.shape, labels.shape, flush=True)
            assert per_token_logp.size(1) >= input_ids.size(1) - 1
            per_token_logp = per_token_logp.tolist()
            # per_token_logp = [x[:input_ids[i].ne(tokenizer.pad_token_id).sum().item()] for i, x in enumerate(per_token_logp)]
            log_prob = log_prob.tolist()
            average_log_prob = average_log_prob.tolist()

            if key == 'win':
                win_logp_list += log_prob
                win_avg_logp_list += average_log_prob
                win_per_token_logp_list += per_token_logp
            else:
                rej_logp_list += log_prob
                rej_avg_logp_list += average_log_prob
                rej_per_token_logp_list += per_token_logp
            # print(f'{key} logits in {output.logits.shape}, logp in {log_prob.shape} avg_logp in {average_log_prob.shape}')

In [ ]:
next(iter(dataloader))

In [ ]:
%debug